In [ ]:
import numpy as np 
import datetime 
import cv2 
from ultralytics import YOLO 

In [ ]:
from deep_sort.deep_sort.tracker import Tracker 
from deep_sort.deep_sort import nn_matching 
from deep_sort.deep_sort.detection import Detection 
from deep_sort.tools import generate_detections as gdet 

In [ ]:
def create_video_writer(video_cap, output_filename):
    # получаем ширину, высоту и частоту кадров видеопотока 
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) 
    fps = int(video_cap.get(cv2.CAP_PROP_FPS)) 
    # инициализируем объект FourCC и видеозаписи 
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') 
    writer = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height)) 
    return writer 

In [ ]:
# задаем некоторые параметры 
conf_threshold = 0.5 
max_cosine_distance = 0.4 
nn_budget = None 

In [ ]:
# Инициализируем объекты захвата видео и записи видео 
video_cap = cv2.VideoCapture("C:\\Users\\bogdan\\jupu\\dip\\videotest1.mp4") 
writer = create_video_writer(video_cap, "outtest1.mp4") 

In [ ]:
# Инициализируем модель YOLOv8 с использованием стандартных весов 
model = YOLO("config/best.pt") 

In [ ]:
# Инициализируем трекер DeepSort 
model_filename = "config/mars-small128.pb" 
encoder = gdet.create_box_encoder(model_filename, batch_size=1) 
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget) 
tracker = Tracker(metric) 

In [ ]:
# загружаем классы COCO, на которых обучена модель YOLO 
classes_path = "config/coco.names" 
with open(classes_path, "r") as f: 
    class_names = f.read().strip().split("\n") 

In [ ]:
# создаем список случайных цветов для каждого класса 
np.random.seed(42)  # чтобы получить одни и те же цвета 
colors = np.random.randint(0, 255, size=(len(class_names), 3))  # (80, 3) 

In [ ]:
# цикл по кадрам видео 
while True: 
    # начальное время для вычисления FPS 
    start = datetime.datetime.now() 
    ret, frame = video_cap.read()  
    # если кадр отсутствует, значит достигнут конец видео 
    if not ret: 
        print("Конец видеофайла...") 
        break 

In [ ]:
############################################################ 
### Обнаружение объектов на кадре с помощью модели YOLO  ### 
############################################################ 

In [ ]:
# запускаем модель YOLO на кадре 
results = model(frame)
# цикл по результатам 
for result in results: 
    # инициализируем списки ограничивающих рамок, уверенностей и идентификаторов классов 
    bboxes = [] 
    confidences = [] 
    class_ids = [] 

In [ ]:
# цикл по обнаруженным объектам 
for data in result.boxes.data.tolist(): 
    x1, y1, x2, y2, confidence, class_id = data 
    x = int(x1) 
    y = int(y1) 
    w = int(x2) - int(x1) 
    h = int(y2) - int(y1) 
    class_id = int(class_id) 
    # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # фильтрация слабых предсказаний, убеждаясь, что уверенность больше минимальной уверенности 
    if confidence > conf_threshold: 
        bboxes.append([x, y, w, h]) 
        confidences.append(confidence) 
        class_ids.append(class_id)

In [ ]:
############################################################ 
### Отслеживание объектов на кадре с помощью DeepSort    ### 
############################################################ 

In [ ]:
# получаем названия обнаруженных объектов 
names = [class_names[class_id] for class_id in class_ids] 
# получаем признаки обнаруженных объектов 
features = encoder(frame, bboxes) 
# преобразуем обнаружения в формат DeepSort 
dets = [] 
for bbox, conf, class_name, feature in zip(bboxes, confidences, names, features): 
    dets.append(Detection(bbox, conf, class_name, feature)) 

In [ ]:
# запускаем трекер на обнаружениях 
    tracker.predict() 
    tracker.update(dets) 

In [ ]:
# цикл по отслеживаемым объектам 
for track in tracker.tracks: 
    if not track.is_confirmed() or track.time_since_update > 1: 
        continue
    # получаем ограничивающую рамку объекта, имя 
    # объекта и идентификатор трека 
    bbox = track.to_tlbr() 
    track_id = track.track_id    
    class_name = track.get_class() 
    # преобразуем ограничивающую рамку в целые числа 
    x1, y1, x2, y2 = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3]) 
    # получаем цвет, соответствующий имени класса 
    class_id = class_names.index(class_name) 
    color = colors[class_id] 
    B, G, R = int(color[0]), int(color[1]), int(color[2])
    # рисуем ограничивающую рамку объекта, имя 
    # предсказанного объекта и идентификатор трека 
    text = str(track_id) + " - " + class_name 
    cv2.rectangle(frame, (x1, y1), (x2, y2), (B, G, R), 2) 
    cv2.rectangle(frame, (x1 - 1, y1 - 20), (x1 + len(text) * 12, y1), (B, G, R), -1) 
    cv2.putText(frame, text, (x1 + 5, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    ############################################################ 
    ### Некоторая постобработка для отображения результатов  ### 
    ############################################################
    # конечное время для вычисления FPS 
    end = datetime.datetime.now() 
    # вычисляем кадры в секунду и выводим на кадр 
    fps = f"FPS: {1 / (end - start).total_seconds():.2f}" 
    cv2.putText(frame, fps, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8) 
    cv2.imshow("Output", frame)
    # записываем кадр на диск 
    writer.write(frame) 
    if cv2.waitKey(1) == ord("q"): 
        break 

In [ ]:
# освобождаем ресурсы, связанные с захватом видео, записью видео и закрываем все окна 
video_cap.release() 
writer.release() 
cv2.destroyAllWindows()